In [2]:
import os

import spotipy
from spotipy.oauth2 import SpotifyOAuth

from config import *

AUTH_URL = "https://accounts.spotify.com/api/token"
API_ENDPOINT = "https://api.spotify.com/v1/"

scope = "user-library-read user-read-recently-played"

sp = spotipy.Spotify(
  auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=scope,
  )
)

In [3]:
from datetime import datetime

datetime.strptime("01/12/2020", "%d/%m/%Y").timestamp()

1606777200.0

In [5]:
# convert timestamp to datetime
datetime.fromtimestamp(1638567430791/1000)

datetime.datetime(2021, 12, 3, 22, 37, 10, 791000)

In [ ]:
from datetime import datetime
history = sp.current_user_recently_played()

In [29]:
history['items']

[{'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7A8S43ryYdbWpJKeHRZRcq'},
      'href': 'https://api.spotify.com/v1/artists/7A8S43ryYdbWpJKeHRZRcq',
      'id': '7A8S43ryYdbWpJKeHRZRcq',
      'name': 'JACKBOYS',
      'type': 'artist',
      'uri': 'spotify:artist:7A8S43ryYdbWpJKeHRZRcq'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/0Y5tJX1MQlPlqiwlOH1tJY'},
      'href': 'https://api.spotify.com/v1/artists/0Y5tJX1MQlPlqiwlOH1tJY',
      'id': '0Y5tJX1MQlPlqiwlOH1tJY',
      'name': 'Travis Scott',
      'type': 'artist',
      'uri': 'spotify:artist:0Y5tJX1MQlPlqiwlOH1tJY'}],
    'available_markets': ['AD',
     'AE',
     'AG',
     'AL',
     'AM',
     'AO',
     'AR',
     'AT',
     'AU',
     'AZ',
     'BA',
     'BB',
     'BD',
     'BE',
     'BF',
     'BG',
     'BH',
     'BI',
     'BJ',
     'BN',
     'BO',
     'BR',
     'BS',
     'BT',
     'BW',
     'BY',
     'BZ

In [17]:
while history['items']:
  print(sp.current_user_recently_played(after=history['items'][-1].keys()))
  history = sp.current_user_recently_played(after=history['items'][-1]['after'])

HTTP Error for GET to https://api.spotify.com/v1/me/player/recently-played with Params: {'limit': 50, 'after': dict_keys(['track', 'played_at', 'context']), 'before': None} returned 400 due to Invalid after cursor


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/me/player/recently-played?limit=50&after=track&after=played_at&after=context:
 Invalid after cursor, reason: None

In [3]:
from collections import defaultdict

from tqdm import tqdm

total = sp.current_user_saved_tracks(limit=1)["total"]
offset = 0
artists_ids_count = defaultdict(int)
tracks = set()
with tqdm(total=245) as pbar:
  while offset < total:
    res = sp.current_user_saved_tracks(limit=50, offset=offset)["items"]
    # get ids of all artists in saved tracks
    for item in res:
      if item["added_at"] < "2021-01-01":
        break
      tracks.add(item["track"]["id"])
      for artist in item["track"]["artists"]:
        artists_ids_count[artist["id"]] += 1
      pbar.update(1)
    offset += 50

100%|██████████| 245/245 [00:02<00:00, 116.82it/s]


In [4]:
# sort by count
sorted_artists = sorted(artists_ids_count.items(), key=lambda x: x[1], reverse=True)
{sp.artist(x)["name"]: count for x, count in sorted_artists}

{'Nina Chuba': 10,
 'NF': 10,
 'ABBA': 7,
 'Billie Eilish': 5,
 'Madison Beer': 5,
 'Apashe': 4,
 'The Tech Thieves': 4,
 'Unlike Pluto': 4,
 'RIELL': 3,
 'blackbear': 3,
 'Sasha Alex Sloan': 3,
 'Besomorph': 3,
 'R3HAB': 3,
 'Ava Max': 2,
 'Tiësto': 2,
 'MEMBA': 2,
 'League of Legends': 2,
 'Yuzmv': 2,
 'Au/Ra': 2,
 'Besomage': 2,
 'Meric Again': 2,
 'Diplo': 2,
 'Mothica': 2,
 'Olivia Rodrigo': 2,
 'Mickey Valen': 2,
 'grandson': 2,
 'Young Franco': 2,
 'Barren Gates': 2,
 'One True God': 2,
 'Sdms': 2,
 'Ranji': 2,
 'Lucky Luke': 2,
 'Ruelle': 2,
 'Stephen': 2,
 'Kim Petras': 2,
 'Ariana Grande': 2,
 'Selena Gomez': 2,
 'why mona': 2,
 'The Kinks': 2,
 'Geoffroy': 2,
 'Laz Kulvic': 1,
 'KLOUD': 1,
 'Cloonee': 1,
 'Adele': 1,
 'Ela Minus': 1,
 'KAYTRANADA': 1,
 'H.E.R.': 1,
 'Erika de Casier': 1,
 'Mura Masa': 1,
 'JACKBOYS': 1,
 'Pop Smoke': 1,
 'Travis Scott': 1,
 'The FifthGuys': 1,
 'Thatsimo': 1,
 'Inova': 1,
 'EVAN GIIA': 1,
 'Ed Sheeran': 1,
 'Tom Enzy': 1,
 'DOVERSTREET': 1,


In [5]:
genres_count = defaultdict(int)
genres_artists = defaultdict(set)
for artist, count in tqdm(artists_ids_count.items()):
  artist_info = sp.artist(artist)
  for g in artist_info["genres"]:
    genres_count[g] += count
    genres_artists[g].add(artist_info["name"])

100%|██████████| 316/316 [00:30<00:00, 10.22it/s]


In [30]:
# sort artists_ids_count by count
sorted_artists = sorted(artists_ids_count.items(), key=lambda x: x[1], reverse=True)
for idx, (artist, count) in enumerate(sorted_artists):
  if count > 1:
    print(sp.artist(artist)["name"], count)

Nina Chuba 10
NF 10
ABBA 7
Billie Eilish 5
Madison Beer 5
Apashe 4
The Tech Thieves 4
Unlike Pluto 4
RIELL 3
blackbear 3
Sasha Alex Sloan 3
Besomorph 3
R3HAB 3
Ava Max 2
Tiësto 2
MEMBA 2
League of Legends 2
Yuzmv 2
Au/Ra 2
Besomage 2
Meric Again 2
Diplo 2
Mothica 2
Olivia Rodrigo 2
Mickey Valen 2
grandson 2
Young Franco 2
Barren Gates 2
One True God 2
Sdms 2
Ranji 2
Lucky Luke 2
Ruelle 2
Stephen 2
Kim Petras 2
Ariana Grande 2
Selena Gomez 2
why mona 2
The Kinks 2
Geoffroy 2


In [26]:
# sort genres_artists with the keys in the same order as genres_count
sorted_genres_artists = sorted(genres_artists.items(), key=lambda x: genres_count[x[0]], reverse=True)

In [27]:
sorted_genres_artists = {k: sorted(v) for k, v in sorted_genres_artists}
sorted_genres_artists

{'pop': ['Adele',
  'Alec Benjamin',
  'Ariana Grande',
  'Au/Ra',
  'Ava Max',
  'Bea Miller',
  'Billie Eilish',
  'Black Eyed Peas',
  'Calvin Harris',
  'Camila Cabello',
  'Cheat Codes',
  'Clairo',
  'Demi Lovato',
  'Dermot Kennedy',
  'Diplo',
  'Dove Cameron',
  'Dua Lipa',
  'Ed Sheeran',
  'Elley Duhé',
  'Frank Ocean',
  'Garrett Nash',
  'Griff',
  'H.E.R.',
  'Imagine Dragons',
  'Jason Derulo',
  'Jaymes Young',
  'Kim Petras',
  'Lana Del Rey',
  'Lily Allen',
  'MEDUZA',
  'Madison Beer',
  'Madonna',
  'Martin Garrix',
  'Michael Jackson',
  'Mothica',
  'Nina Nesbitt',
  'Olivia Rodrigo',
  'PARTYNEXTDOOR',
  'PVRIS',
  'R3HAB',
  'Rihanna',
  'Ruelle',
  'Sasha Alex Sloan',
  'Selena Gomez',
  'Shakira',
  'Tate McRae',
  'Tiësto',
  'Tove Lo',
  'UPSAHL',
  'Zedd',
  'blackbear',
  'bülow',
  'Ängie'],
 'dance pop': ['Afrojack',
  'Ariana Grande',
  'Ava Max',
  'Bea Miller',
  'Black Eyed Peas',
  'Calvin Harris',
  'Camila Cabello',
  'Cheat Codes',
  'DEV',
  'D

In [28]:
import json
# write sorted_genres to json file with utf-8 encoding
with open("genres.json", "w", encoding="utf-8") as f:
  json.dump(sorted_genres_artists, f, ensure_ascii=False, indent=2)

In [29]:
# sort genres by frequency
sorted_genres_count = sorted(genres_count.items(), key=lambda x: x[1], reverse=True)
sorted_genres_count

[('pop', 77),
 ('dance pop', 47),
 ('alt z', 46),
 ('electropop', 35),
 ('bass trap', 32),
 ('edm', 30),
 ('pop rap', 28),
 ('traprun', 25),
 ('pop dance', 24),
 ('electro house', 23),
 ('tropical house', 22),
 ('indie poptimism', 22),
 ('rap', 19),
 ('slap house', 18),
 ('post-teen pop', 17),
 ('hip hop', 17),
 ('pop edm', 15),
 ('house', 14),
 ('modern alternative rock', 13),
 ('modern rock', 13),
 ('rock', 13),
 ('art pop', 10),
 ('hamburg hip hop', 10),
 ('swedish pop', 9),
 ('social media pop', 8),
 ('europop', 7),
 ('electronic trap', 6),
 ('progressive electro house', 6),
 ('gaming edm', 6),
 ('uk pop', 5),
 ('escape room', 5),
 ('indie electropop', 5),
 ('album rock', 5),
 ('classic rock', 5),
 ('uk dance', 5),
 ('psychedelic trance', 5),
 ('progressive house', 5),
 ('la pop', 5),
 ('big room', 4),
 ('dutch edm', 4),
 ('indie soul', 4),
 ('vapor soul', 4),
 ('future bass', 4),
 ('vapor twitch', 4),
 ('dutch house', 4),
 ('sad rap', 4),
 ('deep groove house', 4),
 ('aussietronic